# Test Notebook Graph

In [1]:
import pandas as pd
from py2neo import Graph
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

# host + port
host = 'http://localhost:7474'

# select database name
db_name = 'modelparallel'

# credentials for API
user = 'neo4j'
passwd = '1234'

# connect to database
call_graph = Graph(host, auth=(user, passwd), name=db_name)

## Test functions here

In [91]:
df_time = call_graph.run("""
    MATCH (t:Measurement)
    RETURN t.point_in_time as time
""").to_data_frame()

time_list = df_time['time'].to_list()
del time_list[-1]

df_tu_hti = call_graph.run("""
    MATCH (:Molecule)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
    RETURN DISTINCT t.transformation_unit as transformation_unit
    """).to_data_frame()

df_tu_pt = call_graph.run("""
    MATCH (:Molecule)-[t:POTENTIAL_TRANSFORMATION]->(:Molecule)
    RETURN DISTINCT t.tu_pt as transformation_unit
    """).to_data_frame()

for ele in time_list:
    transform_count_hti = call_graph.run("""
    MATCH (m:Measurement)-[:MEASURED_IN]-(:Molecule)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
    WHERE m.point_in_time = """ + str(ele) + """
    RETURN t.transformation_unit as transformation_unit, count(t.transformation_unit) as Count_HTI_""" + str(ele) + """
    ORDER BY Count_HTI_""" + str(ele) + """ DESC
    """).to_data_frame()

    transform_count_cti = call_graph.run("""
    MATCH (m:Measurement)-[:MEASURED_IN]-(:Molecule)-[t:POTENTIAL_TRANSFORMATION]->(:Molecule)
    WHERE m.point_in_time = """ + str(ele) + """
    RETURN t.tu_pt as transformation_unit, count(t.tu_pt) as Count_PT_""" + str(ele) + """
    ORDER BY Count_PT_""" + str(ele) + """ DESC
    """).to_data_frame()
    
    df_tu_hti = pd.merge(df_tu_hti, transform_count_hti, on=["transformation_unit"])
    df_tu_hti['share_hti_' + str(ele)] = df_tu_hti['Count_HTI_' + str(ele)]/df_tu_hti['Count_HTI_' + str(ele)].sum()*100
    
    df_tu_pt = pd.merge(df_tu_pt, transform_count_cti, on=["transformation_unit"])
    df_tu_pt['share_pt_' + str(ele)] = df_tu_pt['Count_PT_' + str(ele)]/df_tu_pt['Count_PT_' + str(ele)].sum()*100

# drop columns 'Count_'
# HTI
droplist_hti = [i for i in df_tu_hti.columns if i.startswith('Count')]
df_tu_hti = df_tu_hti.drop(columns=droplist_hti, axis=1)
# PT
droplist_pt = [i for i in df_tu_pt.columns if i.startswith('Count')]
df_tu_pt = df_tu_pt.drop(columns=droplist_pt, axis=1)

# make dataframe vertical
# HTI
df_tu_hti = df_tu_hti.replace('', np.nan).set_index('transformation_unit').stack().reset_index(name='share').drop('level_1',1)
# PT
df_tu_pt = df_tu_pt.replace('', np.nan).set_index('transformation_unit').stack().reset_index(name='share').drop('level_1',1)

# add time to dataframe
# HTI
times_repeat = len(df_tu_hti)/len(time_list)
times_list_hti = time_list * int(times_repeat)
df_tu_hti['point_in_time'] = times_list_hti
# PT
times_repeat = len(df_tu_pt)/len(time_list)
times_list_pt = time_list * int(times_repeat)
df_tu_pt['point_in_time'] = times_list_pt

# create plots
# HTI
fig = px.line(df_tu_hti, x='point_in_time', y='share', color='transformation_unit', symbol="transformation_unit",
                labels={
                    "share": "share in %",
                    "point_in_time": "measurement"
                 },
                title="Transformations and their share per measurement - HTI")
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

# PT
fig = px.line(df_tu_pt, x='point_in_time', y='share', color='transformation_unit', symbol="transformation_unit",
                labels={
                    "share": "share in %",
                    "point_in_time": "measurement"
                 },
                title="Transformations and their share per measurement - PT")
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

C:\Users\philipp\AppData\Local\Temp/ipykernel_2928/1852821016.py:50: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

C:\Users\philipp\AppData\Local\Temp/ipykernel_2928/1852821016.py:52: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

